In [2]:
import io         # used to create file streams
import fcntl      # used to access I2C parameters like addresses
import time       # used for sleep delay and timestamps
import string     # helps parse strings
from sense_hat import SenseHat
import subprocess
import re

class AtlasI2C:
    long_timeout = 1.5 # the timeout needed to query readings and calibrations
    short_timeout = .5 # timeout for regular commands
    default_bus = 1 # the default bus for I2C on the newer Raspberry Pis, certain older boards use bus 0
    default_address = 100 # the default address for the sensor
    current_addr = default_address
    
    def __init__(self, address=default_address, bus=default_bus):
        self.file_read = io.open("/dev/i2c-"+str(bus), "rb", buffering=0)
        self.file_write = io.open("/dev/i2c-"+str(bus), "wb", buffering=0)
        self.set_i2c_address(address)

    def set_i2c_address(self, addr):
        I2C_SLAVE = 0x703
        fcntl.ioctl(self.file_read, I2C_SLAVE, addr)
        fcntl.ioctl(self.file_write, I2C_SLAVE, addr)
        self.current_addr = addr

    def write(self, cmd):
        cmd += "\00"
        self.file_write.write(bytes(cmd, 'UTF-8'))

    def read(self, num_of_bytes=32):
        res = self.file_read.read(num_of_bytes)         # read from the board
        response = list(filter(lambda x: x != '\x00', res))    # remove the null characters to get the response
        check_err = response[0]
        if check_err == 1:            
            char_list = map(lambda x: chr(x & ~0x80), list(response[1:]))
            return ''.join(char_list) 
        else:
            return "Error " + str(check_err)

    def query(self, string):
        # write a command to the board, wait the correct timeout, and read the response
        self.write(string)
        #always just do long timeout for simplicity
        time.sleep(self.long_timeout)
        return self.read()

    def close(self):
        self.file_read.close()
        self.file_write.close()
        
    def list_i2c_devices(self):
        prev_addr = self.current_addr # save the current address so we can restore it after
        i2c_devices = []
        for i in range (0,128):
            try:
                self.set_i2c_address(i)
                self.read()
                i2c_devices.append(i)
            except IOError:
                pass
        self.set_i2c_address(prev_addr) # restore the address we were using
        return i2c_devices

In [ ]:
#Setup for microscope and piSense
#modprobe i2c-dev #sensehat
#cd /usr/src/app/RPi_Cam_Web_Interface && service nginx restart #microscope server
#modprobe bcm2835-v4l2 #raspicam

In [2]:
#Start and calibrate devices
device = AtlasI2C() # creates the I2C port object, specify the address or bus if necessary",
delaytime = AtlasI2C.long_timeout

#Setup and maintenance for pH and EC
devices = device.list_i2c_devices()
device.set_i2c_address(99)
device.query("CAL,mid,7.0") #calibrate pH
#device.set_i2c_address(100)
#device.query("Cal,dry") #calibrate E.C. Dry
#device.query("Cal,low,1413") #calibrate E.C. low/single
device.close()

'Error 254'

In [3]:
#start Pisense
sense = SenseHat()


In [4]:
for i in range(0,1):
    t = sense.get_temperature()
    cpu_temp = str((str(subprocess.check_output("vcgencmd measure_temp", shell=True)).replace("temp=","")).replace("'C",""))
    cpu_temp = float(re.findall("\d+\.\d+", cpu_temp)[0])
    t_c = t - ((cpu_temp - t)/1.166) # calibrate according to CPU temp
    t_c = ((t_c/5.0)*9)+32 #farenheit
    p = sense.get_pressure()
    h = sense.get_humidity()
    t_c = round(t_c, 1)
    p = round(p, 1)
    h = round(h, 1)
    
    #Start, query, and shutdown Atlas devices to avoid crosstalk
    device = AtlasI2C()
    #query tentacle shield: 99 = pH, 100 = E.C.
    device.set_i2c_address(99)
    pH = round(float(device.query("R").rstrip("\x00")),2)
    device.set_i2c_address(100)
    ec = device.query("R").rstrip("\x00")
    device.close()
    
    msg = "Euplotid says: Temp = {0}'F, Press = {1}Pa, Humidity = {2}%, pH = {3}, E.C. = {4}".format(t_c,p,h,pH,ec)
    sense.show_message(msg, scroll_speed=0.02)
#    time.sleep(delaytime)